# Bulk Delete Groups by Name Matching

This notebook searches for and deletes multiple Analysis Groups from Moody's Risk Modeler by name pattern.

**Use Case:** Clean up test groups, remove obsolete groups, or clear groups matching a naming pattern.

**Process:**
1. Enter a search pattern (e.g., "test", "Q3_2024")
2. Search for groups whose name CONTAINS the pattern
3. Review the list of matching groups
4. Confirm deletion
5. Delete groups one by one with progress tracking
6. View summary of results

**Warning:** This action permanently deletes groups from Moody's and CANNOT be undone!

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from helpers import ux
from helpers.irp_integration import IRPClient
from helpers.irp_integration.exceptions import IRPAPIError

# Initialize IRP client
irp_client = IRPClient()

ux.header("Bulk Delete Groups")
ux.info("This tool searches for and deletes Analysis Groups from Moody's by name pattern.")

## 1. Search for Groups

Enter a search value to find groups. The search will find all groups whose name **contains** the search value (case-sensitive).

In [ ]:
ux.subheader("Enter Search Pattern")

# Get search value from user
search_value = ux.text_input(
    prompt="Enter search pattern",
    placeholder="test",
    validation=lambda x: len(x.strip()) >= 2  # Minimum 2 characters
)

if search_value is None:
    ux.warning("Operation cancelled by user")
    raise Exception("User cancelled operation")

ux.info(f"Searching for groups containing: '{search_value}'")
print()

# Search for groups using LIKE filter with wildcards + engineType filter
try:
    filter_str = f'analysisName LIKE \"*{search_value}*\" AND engineType = "Group"'
    
    ux.info("Querying Moody's API...")
    groups = irp_client.analysis.search_analyses_paginated(filter=filter_str)
    
    if not groups:
        ux.warning(f"No groups found matching pattern: '{search_value}'")
        print()
        ux.info("Tips:")
        ux.info("  - Check the spelling and case (search is case-sensitive)")
        ux.info("  - Try a shorter/broader search pattern")
        ux.info("  - Verify groups exist in Moody's Risk Modeler UI")
        raise Exception("No groups found")
    
    ux.success(f"Found {len(groups)} group(s) matching pattern")
    print()
    
except IRPAPIError as e:
    ux.error(f"API Error: {e}")
    raise

## 2. Review Groups to Delete

Review the list of groups that will be deleted. Verify these are the correct groups before proceeding.

In [ ]:
ux.subheader("Groups Found")

# Warn if large result set
if len(groups) > 50:
    ux.warning(f"Large result set: {len(groups)} groups found")
    ux.warning("Consider using a more specific search pattern to narrow results")
    if not ux.yes_no("Continue with this large set?"):
        raise Exception("User chose to refine search")
    print()

# Build DataFrame for display
groups_data = []
for g in groups:
    groups_data.append({
        'Analysis ID': g.get('analysisId'),
        'App Analysis ID': g.get('appAnalysisId'),
        'Group Name': g.get('analysisName'),
        'Peril': g.get('perilCode', 'N/A'),
        'Region': g.get('regionCode', 'N/A'),
        'Framework': g.get('analysisFramework', 'N/A')
    })

groups_df = pd.DataFrame(groups_data)
ux.dataframe(groups_df, title=f"Groups matching '{search_value}' ({len(groups_df)} total)")

# Store for deletion step
groups_to_delete = groups.copy()

print()
ux.warning(f"The above {len(groups_to_delete)} group(s) will be PERMANENTLY DELETED from Moody's.")

## 3. Confirm Deletion

**This action cannot be undone!** Please review the groups listed above carefully before confirming.

In [ ]:
ux.header("CONFIRMATION REQUIRED")
ux.error(f"You are about to PERMANENTLY DELETE {len(groups_to_delete)} group(s)")
print()

# First confirmation
if not ux.yes_no("Are you sure you want to delete these groups?"):
    ux.info("Operation cancelled by user")
    raise Exception("User cancelled operation")

# Second confirmation with count
if not ux.yes_no(f"Final confirmation: DELETE {len(groups_to_delete)} group(s)?"):
    ux.info("Operation cancelled by user")
    raise Exception("User cancelled operation")

ux.success("Confirmation received. Starting deletion...")
print()

# Track results
deleted_count = 0
failed_count = 0
errors = []

ux.subheader("Deletion Progress")

for i, group in enumerate(groups_to_delete, 1):
    group_name = group.get('analysisName', 'Unknown')
    analysis_id = group.get('analysisId')
    
    try:
        # Delete the group
        irp_client.analysis.delete_analysis(analysis_id)
        deleted_count += 1
        ux.success(f"  [{i}/{len(groups_to_delete)}] Deleted: {group_name}")
        
    except Exception as e:
        failed_count += 1
        error_msg = f"{group_name}: {str(e)}"
        errors.append(error_msg)
        ux.error(f"  [{i}/{len(groups_to_delete)}] FAILED: {group_name}")
        ux.warning(f"      Error: {str(e)[:100]}")

print()

## 4. Deletion Summary

In [ ]:
ux.header("Deletion Summary")

# Summary statistics
summary_data = [
    ["Search Pattern", search_value],
    ["Groups Found", len(groups_to_delete)],
    ["Successfully Deleted", deleted_count],
    ["Failed", failed_count],
]

ux.table(summary_data, headers=["Metric", "Value"])
print()

# Overall status
if failed_count == 0:
    ux.success(f"All {deleted_count} group(s) deleted successfully!")
elif deleted_count == 0:
    ux.error(f"All {failed_count} deletion(s) failed")
else:
    ux.warning(f"Partial success: {deleted_count} deleted, {failed_count} failed")

# Show errors if any
if errors:
    print()
    ux.subheader("Errors")
    for error in errors:
        ux.error(f"  - {error}")

print()
ux.info("Note: Deleted groups cannot be recovered. If you need to recreate them,")
ux.info("you will need to re-run the grouping workflow.")

---